In [4]:
import qiime2
import os
os.getcwd()
#change working dir to above
os.chdir('/home/rtsantos3/bioinfo_pipelines/picrust2-2.4.2/data/Kiara-Qiime2/')

In [15]:
!qiime tools import   \
--type 'SampleData[PairedEndSequencesWithQuality]'   \
--input-path 'read-files/sample-manifest.tsv'   \
--output-path paired-end-demux.qza   \
--input-format PairedEndFastqManifestPhred33V2 \



There was a problem importing read-files/sample-manifest.tsv:

  Importing 'SingleLanePerSamplePairedEndFastqDirFmt' requires a directory, not read-files/sample-manifest.tsv



In [ ]:
#Denoising Step

!qiime dada2 denoise-single \
    --i-demultiplexed-seqs paired-end-demux.qza \
    --p-trunc-len 140 \
    --p-n-threads 2 \
    --output-dir dada2 \ 
    --verbose

In [32]:
#Denoising Step (paired )

!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs paired-end-demux.qza \
    --p-trunc-len-f 300 \
    --p-trunc-len-r 300 \
    --p-n-threads 2 \
    --output-dir "paired/dada2-paired" \
    --verbose

#Analyzing paired sequences does not produce any proper results. Bakit kaya?
#Oks na I just needed to keep lengths at around 300


Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmp1x64likz/forward /tmp/tmp1x64likz/reverse /tmp/tmp1x64likz/output.tsv.biom /tmp/tmp1x64likz/track.tsv /tmp/tmp1x64likz/filt_f /tmp/tmp1x64likz/filt_r 300 300 0 0 2.0 2.0 2 12 independent consensus 1.0 2 1000000

R version 4.0.5 (2021-03-31) 
Loading required package: Rcpp
DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering ....
2) Learning Error Rates
41432700 total bases in 138109 reads from 4 samples will be used for learning the error rates.
41432700 total bases in 138109 reads from 4 samples will be used for learning the error rates.
3) Denoise samples ....
....
4) Remove chimeras (method = consensus)
6) Write output
Saved FeatureTable[Frequency] to: paired/dada2-paired/table.qza
Saved FeatureData[Sequence

In [20]:
!qiime feature-classifier fit-classifier-naive-bayes \
        --i-reference-reads "../training-sets/silva-138-99-seqs.qza" \
        --i-reference-taxonomy "../training-sets/silva-138-99-tax.qza" \
        --o-classifier "TaxonomicClassifier.qza" \
        --output-dir "../training-sets/Classifier" \
        --verbose 

/home/rtsantos3/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/q2_feature_classifier/classifier.py:102: UserWarning: The TaxonomicClassifier artifact that results from this method was trained using scikit-learn version 0.24.1. It cannot be used with other versions of scikit-learn. (While the classifier may complete successfully, the results will be unreliable.)
  warnings.warn(warning, UserWarning)


In [ ]:
!qiime feature-classifier classify-consensus-vsearch \
        --i-query "../dada2/representative_sequences.qza" \
        --i-reference-reads "../training-sets/silva-138-99-seqs.qza" \
        --i-reference-taxonomy "../training-sets/silva-138-99-tax.qza" \
        --output-dir "../taxonomy_vsearch" 

In [34]:
#Vsearch for dada2-paired

!qiime feature-classifier classify-consensus-vsearch \
        --i-query "paired/dada2-paired/representative_sequences.qza" \
        --i-reference-reads "../training-sets/silva-138-99-seqs.qza" \
        --i-reference-taxonomy "../training-sets/silva-138-99-tax.qza" \
        --output-dir "paired/taxonomy_vsearch_paired" 

Saved FeatureData[Taxonomy] to: paired/taxonomy_vsearch_paired/classification.qza


In [ ]:
!qiime feature-classifier classify-sklearn --help \
        --i-reads "../dada2/representative_sequences.qza" \
        --i-classifier "../training-sets/"

In [28]:
# !qiime tools export \
#     --input-path "/taxonomy_vsearch/classification.qza" \
#     --output-path "../taxonomy_vsearch/"

#Exporting paired 
!qiime tools export \
    --input-path "paired/taxonomy_vsearch_paired/classification.qza" \
    --output-path "paired/taxonomy_vsearch_paired"

Exported paired/taxonomy_vsearch_paired/classification.qza as TSVTaxonomyDirectoryFormat to directory paired/taxonomy_vsearch_paired


In [ ]:
!qiime tools export \
    --input-path "../pai/taxonomy_vsearch/classification.qza" \
    --output-path "../taxonomy_vsearch/"

In [9]:
!qiime metadata tabulate \
    --m-input-file dada2/denoising_stats.qza \
    --o-visualization dada2/denoising-stats.qzv

Saved Visualization to: dada2/denoising-stats.qzv


In [13]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences /dada2/representative_sequences.qza \
    --output-dir tree

Saved FeatureData[AlignedSequence] to: tree/alignment.qza
Saved FeatureData[AlignedSequence] to: tree/masked_alignment.qza
Saved Phylogeny[Unrooted] to: tree/tree.qza
Saved Phylogeny[Rooted] to: tree/rooted_tree.qza


In [9]:
#This code block is to filter out any Eukaryotic sequences from the feature table
!qiime taxa filter-table \
  --i-table './dada2/table.qza' \
  --i-taxonomy './taxonomy_vsearch/classification.qza' \
  --p-mode exact \
  --p-exclude "d__Eukaryota; p__Ascomycota; c__Dothideomycetes; o__Capnodiales; f__Mycosphaerellaceae" \
  --o-filtered-table './dada2/no_eukaryotes_filtered_table.qza'



Saved FeatureTable[Frequency] to: ./dada2/no_eukaryotes_filtered_table.qza


In [2]:
#This code block is to filter out any Eukaryotic sequences from the feature table (Paired)
!qiime taxa filter-table \
  --i-table '../paired/dada2-paired/table.qza' \
  --i-taxonomy '../paired/taxonomy_vsearch_paired/classification.qza' \
  --p-mode exact \
  --p-exclude "d__Eukaryota" \
  --o-filtered-table '../paired/dada2-paired/no_eukaryotes_filtered_table.qza'



Saved FeatureTable[Frequency] to: ../paired/dada2-paired/no_eukaryotes_filtered_table.qza


In [9]:
#For validating paired


!qiime taxa barplot  \
    --i-table './paired/dada2-paired/no_eukaryotes_filtered_table.qza' \
    --i-taxonomy 'paired/taxonomy_vsearch_paired/classification.qza' \
    --m-metadata-file './read-files/metadata.tsv' \
    --o-visualization './paired/plots/qiime_taxa_barplot/visualization' \
    --verbose 

Saved Visualization to: ./paired/plots/qiime_taxa_barplot/visualization.qzv


These code-blocks below is to compute both the Alpha and Beta Rarefaction curves as well as other diversity stats using the filtered  table

In [11]:
#Alpha Diversity stat

!qiime diversity core-metrics-phylogenetic \
    --i-table './dada2/no_eukaryotes_filtered_table.qza' \
    --i-phylogeny './tree/rooted_tree.qza' \
    --p-sampling-depth 10000 \
    --m-metadata-file './read-files/metadata.tsv' \
    --output-dir diversity

Saved FeatureTable[Frequency] to: diversity/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: diversity/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: diversity/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: diversity/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: diversity/evenness_vector.qza
Saved DistanceMatrix to: diversity/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: diversity/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: diversity/jaccard_distance_matrix.qza
Saved DistanceMatrix to: diversity/bray_curtis_distance_matrix.qza
Saved PCoAResults to: diversity/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: diversity/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: diversity/jaccard_pcoa_results.qza
Saved PCoAResults to: diversity/bray_curtis_pcoa_results.qza
Saved Visualization to: diversity/unweighted_unifrac_emperor.qzv
Saved Visualization to: diversity/weighted_unifrac_emperor.qzv
Saved

In [12]:
!qiime diversity alpha-rarefaction \
    --i-table './dada2/no_eukaryotes_filtered_table.qza' \
    --i-phylogeny './tree/rooted_tree.qza' \
    --p-max-depth 4970 \
    --p-metrics 'shannon' 'fisher_alpha' 'chao1' 'goods_coverage' 'simpson_e' 'observed_features' \
    --m-metadata-file './read-files/metadata.tsv' \
    --output-dir './diversity/alpha-rarefaction/' \
    --verbose

Saved Visualization to: ./diversity/alpha-rarefaction/visualization.qzv


In [14]:
!qiime diversity beta-rarefaction \
    --i-table './dada2/no_eukaryotes_filtered_table.qza' \
    --i-phylogeny './tree/rooted_tree.qza' \
    --p-metric 'weighted_unifrac' \
    --p-clustering-method 'upgma' \
    --m-metadata-file './read-files/metadata.tsv' \
    --p-sampling-depth 4970 \
    --p-correlation-method 'spearman' \
    --output-dir './diversity/beta-diversity' \
    --verbose

Saved Visualization to: ./diversity/beta-diversity/visualization.qzv


In [14]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

Saved Visualization to: tree/empress.qzv


In [26]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file 'read-files/metadata.tsv' \
    --o-visualization diversity/alpha_groups.qzv

Saved Visualization to: diversity/alpha_groups.qzv


In [39]:
#Permanova via Adonis addon

!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file read-files/metadata.tsv \
    --p-formula "treatment" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

Saved Visualization to: diversity/permanova.qzv


In [26]:
#Running Picrust2 module on ASVs obtained as well as on the representative sequences
#Not needed na kasi I ran the standalone version, which is capable of 
!qiime picrust2 full-pipeline\
    --i-table dada2/table.qza \
    --i-seq dada2/representative_sequences.qza\
    --output-dir q2-picrust2 \
    --p-placement-tool sepp \
    --p-threads 6 \
    --p-hsp-method pic \
    --p-max-nsti 2 \
    --verbose

Warning - 1 input sequences aligned poorly to reference sequences (--min_align option specified a minimum proportion of 0.8 aligning to reference sequences). These input sequences will not be placed and will be excluded from downstream steps.

This is the set of poorly aligned input sequences to be excluded: 7e879359d8a9e9b3c08b121533609d54





All ASVs were below the max NSTI cut-off of 2.0 and so all were retained for downstream analyses.

All ASVs were below the max NSTI cut-off of 2.0 and so all were retained for downstream analyses.


Saved FeatureTable[Frequency] to: q2-picrust2/ko_metagenome.qza
Saved FeatureTable[Frequency] to: q2-picrust2/ec_metagenome.qza
Saved FeatureTable[Frequency] to: q2-picrust2/pathway_abundance.qza


In [27]:
#Generate picrust2 feature table
!qiime feature-table summarize \
    --i-table q2-picrust2/pathway_abundance.qza \
    --o-visualization q2-picrust2/pathway_abundance.qzv 

Saved Visualization to: q2-picrust2/pathway_abundance.qzv


In [28]:
#Export pathway tables to biom format

!qiime tools export \
    --input-path q2-picrust2/pathway_abundance.qza \
    --output-path pathabun_exported 




Exported q2-picrust2/pathway_abundance.qza as BIOMV210DirFmt to directory pathabun_exported


In [7]:
#Export KO table to biom format

!qiime tools export \
    --input-path q2-picrust2/ko_metagenome.qza \
    --output-path ko_metagenome

!biom convert \
    -i ko_metagenome/ko-feature-table.biom \
    -o ko_metagenome/ko-feature-table.biom.tsv \
    --to-tsv

Exported q2-picrust2/ko_metagenome.qza as BIOMV210DirFmt to directory ko_metagenome


In [6]:
#Export EC table to biom format

!qiime tools export \
    --input-path q2-picrust2/ec_metagenome.qza \
    --output-path ec_metagenome

!biom convert \
    -i ec_metagenome/ec-feature-table.biom \
    -o ec_metagenome/ec-feature-table.biom.tsv \
    --to-tsv

Exported q2-picrust2/ec_metagenome.qza as BIOMV210DirFmt to directory ec_metagenome


In [29]:
!biom convert \
    -i pathabun_exported/feature-table.biom \
    -o pathabun_exported/feature-table.biom.tsv \
    --to-tsv

In [5]:
#Convert DADA2 output to fasta and tsv, respectively

!qiime tools export \
    --input-path dada2/table.qza \
    --output-path Stratified-pipeline/

!qiime tools export \
    --input-path dada2/representative_sequences.qza \
    --output-path Stratified-pipeline/rep_seqs.fasta


Exported dada2/table.qza as BIOMV210DirFmt to directory Stratified-pipeline/
Exported dada2/representative_sequences.qza as DNASequencesDirectoryFormat to directory Stratified-pipeline/rep_seqs.fasta
